# Stable Diffusion による画像の生成

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

---

このデモノートブックでは、[Amazon Bedrock](https://aws.amazon.com/bedrock/) で [Stable Diffusion XL](https://stability.ai/stablediffusion) (SDXL) を使用して画像生成 (Text to Image) と画像編集 (画像から画像へ) を行う方法を示します。

Stable Diffusion の画像は、以下の4つの主要モデルによって生成されます
1. CLIP テキストエンコーダー
2. VAE デコーダー
3. UNet
4. VAE_Post_Quant_Conv

これらのブロックが選ばれたのは、パイプライン内のコンピューティングの大部分を占めるからです。

下の図を参照してください

![SD アーキテクチャ](./images/sd.png)

### 画像プロンプト

良いプロンプトを書くことは、いくぶん芸術のようなものです。特定のプロンプトが特定のモデルで満足のいく画像を生成するかどうかを予測することはしばしば困難です。ただし、機能することが確認されている特定のテンプレートがあります。プロンプトは大きく分けて次の 3 つに分けることができます。

1. 画像の**タイプ**（写真/スケッチ/絵画など）
2. コンテンツ（件名、オブジェクト、環境、シーンなど）の**説明**
3. 画像の**スタイル**（現実的/芸術的/芸術的/芸術の種類など）

3 つのパーツをそれぞれ個別に変更して、画像のバリエーションを生成できます。形容詞は、画像生成プロセスにおいて重要な役割を果たすことが知られています。また、詳細を追加すると生成プロセスに役立ちます。

リアルな画像を生成するには、“a photo of”, “a photograph of”, “realistic”, “hyper realistic” などのフレーズを使用できます。アーティストによる画像を生成するには、「by Pablo Picasso（パブロ・ピカソ作）」、「oil painting by Rembrandt（レンブラントの油絵）」、「landscape art by Frederic Edwin Church（フレデリック・エドウィン・チャーチの風景画）」、「pencil drawing by Albrecht Dürer（アルブレヒト・デューラーの鉛筆画）」などのフレーズを使用できます。異なるアーティストを組み合わせることも可能です。アートカテゴリー別に芸術的なイメージを生成するには、プロンプトに「lion on a beach, abstract（浜辺のライオン、抽象）」のようにアートカテゴリーを追加します。その他のカテゴリーには、「oil painting（油絵）」、「pencil drawing（鉛筆画）」、「pop art（ポップアート）」、「digital art（デジタルアート）」、「anime（アニメ）」、「cartoon（カートゥーン）」、「futurism（未来派）」、「watercolor（水彩）」、「manga（マンガ）」などがあります。また、「35mm wide lens（35 mm ワイドレンズ）」や「85 mm wide lens（85 mm ワイドレンズ）」などの照明やカメラレンズの詳細や、フレーミングに関する詳細（portrait/landscape/close up（ポートレート/風景/クローズアップ）など）を含めることもできます。

同じプロンプトが複数回与えられても、モデルは異なる画像を生成することに注意してください。そのため、複数の画像を生成して、アプリケーションに最適な画像を選択できます。

## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrockに接続する必要があります。

セットアップの仕組みと ⚠️ **変更が必要かどうか**についての詳細は、[Bedrock boto3 セットアップノートブック](../00_Intro/Bedrock_boto3_Setup.ja.ipynb) ノートブックを参照してください。

このノートブックには、画像を操作するための [Pillow](https://pillow.readthedocs.io/en/stable/) もインストールする必要があります。

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet "pillow>=9.5,<10"

In [ ]:
# Python ビルトイン:
import base64
import io
import json
import os
import sys

# 外部依存関係:
import boto3
from PIL import Image

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ コメントを外して、AWS の設定に応じて以下の行を編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## Text to Image 

Text to Image モードでは、 `プロンプト`と呼ばれる、どの画像を生成**すべきか**というテキストによる説明を提供します。

Stable Diffusion XL (SDXL) では、生成に影響を与えるのに役立つ特定の [スタイルプリセット](https://platform.stability.ai/docs/release-notes#style-presets) を指定することもできます。

しかし、特定のコンテンツやスタイルの選択を***避ける***ようにモデルに働きかけたい場合はどうでしょうか。画像生成モデルは通常、*画像の説明*からトレーニングされるため、プロンプトで**望まない**ものを直接指定しようとしても（たとえば、「あごひげを生やしていない男」）、通常はうまくいきません。画像をそうでないもので記述することは非常に珍しいことです。

代わりに、SDXLでは各プロンプトに `重み`を指定できますが、これは負の値でもかまいません。これを使用して、以下に示すように `ネガティブプロンプト`を提供します :

In [ ]:
prompt = "Dog in a forest"
negative_prompts = [
    "poorly rendered",
    "poor background details",
    "poorly drawn dog",
    "disfigured dog features",
]
style_preset = "photographic"  # (e.g. photographic, digital-art, cinematic, ...)
#prompt = "photo taken from above of an italian landscape. cloud is clear with few clouds. Green hills and few villages, a lake"

Amazon Bedrock `InvokeModel` は、適切なモデル ID を設定することで SDXL へのアクセスを提供し、(PNG) イメージを表す [Base64 エンコードされた文字列](https://en.wikipedia.org/wiki/Base64) を含む JSON レスポンスを返します。

モデルで使用可能な入力パラメーターの詳細については、[Stability AI ドキュメント](https://platform.stability.ai/docs/api-reference#tag/v1generation/operation/textToImage) を参照してください。

以下のセルは Amazon Bedrock を通じて SDXL モデルを呼び出し、初期画像文字列を作成します :

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": 5,
    "seed": 5450,
    "steps": 70,
    "style_preset": style_preset,
})
modelId = "stability.stable-diffusion-xl"

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

Base64 文字列をバイナリにデコードし、PNG ファイルを読み取ることができる [Pillow](https://pillow.readthedocs.io/en/stable/) のような画像処理ライブラリにロードすることで、ノートブック内の画像を表示したり操作したりできます :

In [ ]:
os.makedirs("data", exist_ok=True)
image_1 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
image_1.save("data/image_1.png")
image_1

## Image to Image

テキストからの画像の生成は強力ですが、場合によっては「正しい」画像を得るために何度もプロンプトの調整が必要になることがあります。

画像間生成では、毎回テキストを一から作成するのではなく、**既存の画像を変更**して、希望する特定の変更を加えることができます。

初期画像を base64 エンコーディングで API に渡す必要があるので、まずそれを準備しましょう。前のセクションの初期画像を使用することも、必要に応じて別の画像を使用することもできます :

In [ ]:
def image_to_base64(img) -> str:
    """Convert a PIL Image or local image file path to a base64 string for Amazon Bedrock"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        print("Converting PIL Image to base64 string")
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")


init_image_b64 = image_to_base64(image_1)
print(init_image_b64[:80] + "...")

新しいガイドプロンプトは、モデルが初期画像に基づいて動作するのに役立ちます

In [ ]:
change_prompt = "add some leaves around the dog"

次に、既存の画像は `init_image` パラメーターを介して Stable Diffusion モデルに渡されます。

繰り返しになりますが、さまざまなパラメーターの使用方法に関するその他のヒントについては、[Stable Diffusion API ドキュメント](https://platform.stability.ai/docs/api-reference#tag/v1generation/operation/imageToImage) を参照してください。

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": change_prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": 10,
    "init_image": init_image_b64,
    "seed": 321,
    "start_schedule": 0.6,
    "steps": 50,
    "style_preset": style_preset,
})
modelId = "stability.stable-diffusion-xl"

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
image_2_b64_str = response_body["artifacts"][0].get("base64")
print(f"{image_2_b64_str[0:80]}...")

In [ ]:
image_2 = Image.open(io.BytesIO(base64.decodebytes(bytes(image_2_b64_str, "utf-8"))))
image_2.save("data/image_2.png")
image_2

## まとめ

このラボでは、[Amazon Bedrock](https://aws.amazon.com/bedrock/) の [Stable Diffusion XL](https://stability.ai/stablediffusion) を使用して、テキストから新しい画像を生成し、既存の画像をテキストの指示で変換する方法を示しました。

Bedrock APIを通じて、画像生成に影響を与えるさまざまなパラメーターを提供できます。これらのパラメーターは、[Stable Diffusion APIドキュメント](https://platform.stability.ai/docs/api-reference#tag/v1generation) に概ね対応しています。

Bedrockを使用する際に注意すべき重要な点の1つは、出力画像 PNG / JPEG データが JSON API 応答内で [Base64 でエンコードされた文字列](https://en.wikipedia.org/wiki/Base64) として返されることです。Pythonの組み込み [base64 ライブラリ](https://docs.python.org/3/library/base64.html) を使用して、この画像データをデコードできます。たとえば、`.png` ファイルを保存する場合などです。また、[Pillow](https://pillow.readthedocs.io/en/stable/) のような画像処理ライブラリを使用して Python 内で画像を読み込み (場合によっては編集) できることも示しました。

ここから、より高度な画像生成オプションを検討したり、生成系 AI を従来の画像処理ツールと組み合わせたりして、ユースケースに最適なクリエイティブワークフローを構築できます。